In [3]:
import json
import requests

In [54]:
data = []
url = "https://api.jcdecaux.com/vls/v1/stations?contract=maribor&apiKey=5e150537116dbc1786ce5bec6975a8603286526b"
response = requests.get(url)
if response.status_code == 200:
    print("Fetched main datset")
    data = json.loads(response.content)
else:
    print("Failed to retrieve JSON data")

Fetched main datset


In [55]:
filtered_data = [record for record in data if record.get('name') == "GOSPOSVETSKA C. - TURNERJEVA UL."]
print(filtered_data)

[{'number': 1, 'contract_name': 'maribor', 'name': 'GOSPOSVETSKA C. - TURNERJEVA UL.', 'address': 'Gosposvetska cesta 69', 'position': {'lat': 46.562695, 'lng': 15.62935}, 'banking': False, 'bonus': False, 'bike_stands': 22, 'available_bike_stands': 9, 'available_bikes': 13, 'status': 'OPEN', 'last_update': 1710015658000}]


In [56]:
import pandas as pd


# Convert the 'position' dictionary into separate 'lat' and 'lng' columns
for record in filtered_data:
    position = record.pop('position')  # Remove 'position' and store its value
    record['lat'] = position['lat']  # Add 'lat' to the record
    record['lng'] = position['lng']  # Add 'lng' to the record
    print(record)

# Create a DataFrame
df = pd.DataFrame(filtered_data)

{'number': 1, 'contract_name': 'maribor', 'name': 'GOSPOSVETSKA C. - TURNERJEVA UL.', 'address': 'Gosposvetska cesta 69', 'banking': False, 'bonus': False, 'bike_stands': 22, 'available_bike_stands': 9, 'available_bikes': 13, 'status': 'OPEN', 'last_update': 1710015658000, 'lat': 46.562695, 'lng': 15.62935}


In [52]:
df

,number,contract_name,name,address,banking,bonus,bike_stands,available_bike_stands,available_bikes,status,last_update,lat,lng
0,1,maribor,GOSPOSVETSKA C. - TURNERJEVA UL.,Gosposvetska cesta 69,False,False,22,9,13,OPEN,1710015658000,46.562695,15.62935


In [57]:
odf = pd.read_csv('../data/raw/mbajk_api.csv', index_col=0)
last_update = df['last_update'].values[0]
if last_update not in odf['last_update'].values:
    odf = pd.concat([odf, df], axis=0).reset_index(drop=True)

odf

,contract_name,name,address,banking,bonus,bike_stands,available_bike_stands,available_bikes,status,last_update,lat,lng,number
0,maribor,GOSPOSVETSKA C. - TURNERJEVA UL.,Gosposvetska cesta 69,False,False,22,10,12,OPEN,1710014510000,46.562695,15.62935,NaN
1,maribor,GOSPOSVETSKA C. - TURNERJEVA UL.,Gosposvetska cesta 69,False,False,22,9,13,OPEN,1710015052000,46.562695,15.62935,NaN
2,maribor,GOSPOSVETSKA C. - TURNERJEVA UL.,Gosposvetska cesta 69,False,False,22,9,13,OPEN,1710015658000,46.562695,15.62935,1.0


In [178]:
df1 = odf[['last_update', 'available_bike_stands', ]].copy()
df1['date'] = pd.to_datetime(df1['last_update'], unit='ms')
df1 = df1.drop(columns='last_update')
df1

,available_bike_stands,date
0,10,2024-03-09 20:01:50
1,9,2024-03-09 20:10:52
2,9,2024-03-09 20:20:58


In [179]:
import math
df1['date_hour'] = df1['date'].dt.floor('h')
result = df1.groupby('date_hour')['available_bike_stands'].mean().reset_index()
result['available_bike_stands'] = result['available_bike_stands'].apply(math.floor)
df1 = result.copy()
df1

,date_hour,available_bike_stands
0,2024-03-09 20:00:00,9


In [180]:

df2 = pd.read_csv('../data/raw/mbajk_dataset.csv')
df2['date'] = pd.to_datetime(df2['date'])
df2['date'] = df2['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2[['date', 'available_bike_stands']]
df2['date_hour'] = df2['date'].dt.floor('h')

result = df2.groupby('date_hour')['available_bike_stands'].mean().reset_index()
result['available_bike_stands'] = result['available_bike_stands'].apply(math.floor)
df2 = result.copy()
df2

,date_hour,available_bike_stands
0,2023-06-25 19:00:00,7
1,2023-06-25 20:00:00,8
2,2023-06-25 21:00:00,4
3,2023-06-25 22:00:00,0
4,2023-06-25 23:00:00,0
...,...,...
2339,2023-10-03 11:00:00,12
2340,2023-10-03 12:00:00,16
2341,2023-10-03 13:00:00,20
2342,2023-10-03 14:00:00,19


In [181]:
df = pd.concat([df2, df1], axis=0).reset_index(drop=True)


In [182]:
df

,date_hour,available_bike_stands
0,2023-06-25 19:00:00,7
1,2023-06-25 20:00:00,8
2,2023-06-25 21:00:00,4
3,2023-06-25 22:00:00,0
4,2023-06-25 23:00:00,0
...,...,...
2340,2023-10-03 12:00:00,16
2341,2023-10-03 13:00:00,20
2342,2023-10-03 14:00:00,19
2343,2023-10-03 15:00:00,12


In [183]:

df['date_hour'] = pd.to_datetime(df['date_hour'])
df['date_hour'] = df['date_hour'].dt.tz_localize(None)
df = df.sort_values(by='date_hour')
df

In [184]:
df

,date_hour,available_bike_stands
0,2023-06-25 19:00:00,7
1,2023-06-25 20:00:00,8
2,2023-06-25 21:00:00,4
3,2023-06-25 22:00:00,0
4,2023-06-25 23:00:00,0
...,...,...
2340,2023-10-03 12:00:00,16
2341,2023-10-03 13:00:00,20
2342,2023-10-03 14:00:00,19
2343,2023-10-03 15:00:00,12


In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.callbacks import CSVLogger

# Load data from CSV file into a DataFrame
def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

# Define model architecture
def build_model(input_shape):
    model = Sequential([
        SimpleRNN(10, input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Load data
file_path = '../data/processed/data.csv'  # Update with your file path
data_df = load_data(file_path)
csv_logger = CSVLogger('../reports/train_metrics.txt')

# Define input sequence length
sequence_length = 10

# Build input sequences and targets
X = []
y = []
for i in range(len(data_df) - sequence_length):
    X.append(data_df['available_bike_stands'].iloc[i:i+sequence_length])
    y.append(data_df['available_bike_stands'].iloc[i+sequence_length])
X = np.array(X)
y = np.array(y)

# Reshape input data
X = X.reshape(-1, sequence_length, 1)

# Build model
input_shape = (sequence_length, 1)
model = build_model(input_shape)

# Train model
model.fit(X, y, epochs=10, batch_size=32, callbacks=[csv_logger])


Epoch 1/10
73/73 [==============================] - 1s 3ms/step - loss: 167.0644
Epoch 2/10
73/73 [==============================] - 0s 3ms/step - loss: 137.8118
Epoch 3/10
73/73 [==============================] - 0s 3ms/step - loss: 121.7990
Epoch 4/10
73/73 [==============================] - 0s 3ms/step - loss: 108.4780
Epoch 5/10
73/73 [==============================] - 0s 2ms/step - loss: 96.7959
Epoch 6/10
73/73 [==============================] - 0s 2ms/step - loss: 86.4704
Epoch 7/10
73/73 [==============================] - 0s 2ms/step - loss: 77.2634
Epoch 8/10
73/73 [==============================] - 0s 3ms/step - loss: 69.0690
Epoch 9/10
73/73 [==============================] - 0s 2ms/step - loss: 61.7448
Epoch 10/10
73/73 [==============================] - 0s 2ms/step - loss: 55.2343


In [3]:
model.save('../models/simple-rnn.h5')

In [2]:
import numpy as np
data = np.array([1,2,3,4,5,6])
data = data.reshape(-1, 1)
data

array([[1],
       [2],
       [3],
       [4],
       [5],
       [6]])

In [1]:
from datetime import datetime

# Get the current date
current_date = datetime.now()

# Format the date to YYYY-MM-DD
formatted_date = current_date.strftime('%Y-%m-%d')

print(formatted_date)


2024-03-11


In [2]:
# Syntax
# variable = value_if_true if condition else value_if_false

# Example
x = 10
y = 20
max_value = x if x > y else y

print(max_value)


20


In [11]:
WEATHER_URL = 'https://api.open-meteo.com/v1/forecast?'
lat = 46.562695
lon = 15.62935

url = WEATHER_URL + f'latitude={lat}&longitude={lon}&'
url += '&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m&forecast_days=1'

response = requests.get(url)
if response.status_code == 200:
    print("Successfully retrieved data")
    data = json.loads(response.content)
else:
    print("Failed to retrieve JSON data")

data = data['hourly']



Successfully retrieved data


In [12]:
import pandas as pd

# Convert the dictionary into a pandas DataFrame
df = pd.DataFrame(data)

# If you want the 'time' column to be a datetime type (optional but often useful)
df['time'] = pd.to_datetime(df['time'])

print(df)

                  time  temperature_2m  relative_humidity_2m  wind_speed_10m
0  2024-03-11 00:00:00             7.7                    92             1.8
1  2024-03-11 01:00:00             7.7                    92             4.1
2  2024-03-11 02:00:00             6.9                    89             3.3
3  2024-03-11 03:00:00             6.4                    90             1.6
4  2024-03-11 04:00:00             5.6                    89             1.9
5  2024-03-11 05:00:00             5.2                    89             3.2
6  2024-03-11 06:00:00             5.1                    92             4.6
7  2024-03-11 07:00:00             5.6                    94             1.6
8  2024-03-11 08:00:00             7.7                    89             2.4
9  2024-03-11 09:00:00             9.2                    85             2.7
10 2024-03-11 10:00:00             9.6                    84             4.3
11 2024-03-11 11:00:00             9.6                    87             6.9

In [13]:
df.to_csv('../data/raw/weather/weather_api.csv')